# Vessel Depressurization Tutorial

This notebook demonstrates NeqSim's vessel filling and depressurization capabilities, inspired by [HydDown](https://github.com/andr1976/HydDown).

## Reference

If you use these capabilities, please cite:

> Andreasen, A. (2021). HydDown: A Python package for calculation of hydrogen (or other gas) pressure vessel filling and discharge. *Journal of Open Source Software*, 6(66), 3695. https://doi.org/10.21105/joss.03695

```bibtex
@article{Andreasen2021, 
  doi = {10.21105/joss.03695}, 
  url = {https://doi.org/10.21105/joss.03695}, 
  year = {2021}, 
  publisher = {The Open Journal}, 
  volume = {6}, 
  number = {66}, 
  pages = {3695}, 
  author = {Anders Andreasen}, 
  title = {HydDown: A Python package for calculation of hydrogen (or other gas) pressure vessel filling and discharge}, 
  journal = {Journal of Open Source Software} 
}
```

## Setup
First, we import the NeqSim Java classes using jpype.

In [ ]:
import jpype
import jpype.imports
from jpype.types import *
import matplotlib.pyplot as plt
import numpy as np

# Start JVM if not already running
if not jpype.isJVMStarted():
    jpype.startJVM(classpath=['../target/classes'])

# Import NeqSim classes
from neqsim.thermo.system import SystemSrkEos, SystemSrkCPAstatoil
from neqsim.process.equipment.stream import Stream
from neqsim.process.equipment.tank import VesselDepressurization
from neqsim.process.util.fire import ReliefValveSizing
from java.util import UUID

print("NeqSim loaded successfully!")

## Example 1: Isothermal Methane Depressurization

This represents a very slow depressurization where the vessel has time to equilibrate with surroundings (constant temperature).

In [ ]:
# Create thermodynamic system with methane at 50 bar, 300 K
gas = SystemSrkEos(300.0, 50.0)
gas.addComponent("methane", 1.0)
gas.setMixingRule("classic")
gas.createDatabase(True)

# Create inlet stream
feed = Stream("feed", gas)
feed.setFlowRate(0.0, "kg/hr")  # Closed vessel
feed.run()

# Create vessel
vessel = VesselDepressurization("CH4 Vessel", feed)
vessel.setVolume(1.0)  # 1 m³
vessel.setOrificeDiameter(0.01)  # 10 mm orifice
vessel.setBackPressure(1.0)  # 1 bar ambient
vessel.setCalculationType(VesselDepressurization.CalculationType.ISOTHERMAL)
vessel.run()

# Run transient simulation for 60 seconds
dt = 0.1
end_time = 60.0
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel.getPressure("bar")]
mass_data = [vessel.getMass()]

for t in np.arange(dt, end_time + dt, dt):
    vessel.runTransient(dt, uuid)
    time_data.append(t)
    pressure_data.append(vessel.getPressure("bar"))
    mass_data.append(vessel.getMass())

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(time_data, pressure_data, 'b-', linewidth=2)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Pressure [bar]')
ax1.set_title('Isothermal Methane Depressurization')
ax1.grid(True)

ax2.plot(time_data, mass_data, 'r-', linewidth=2)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Mass [kg]')
ax2.set_title('Mass Remaining in Vessel')
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"Initial: P = 50.0 bar, mass = {mass_data[0]:.2f} kg")
print(f"Final:   P = {pressure_data[-1]:.1f} bar, mass = {mass_data[-1]:.2f} kg")
print(f"Mass discharged: {mass_data[0] - mass_data[-1]:.2f} kg ({100*(1 - mass_data[-1]/mass_data[0]):.1f}%)")

## Example 2: Isentropic Nitrogen Depressurization

Adiabatic depressurization with PV work - the gas cools as it expands.

In [ ]:
# Create nitrogen system at 50 bar, 300 K
n2 = SystemSrkEos(300.0, 50.0)
n2.addComponent("nitrogen", 1.0)
n2.setMixingRule("classic")
n2.createDatabase(True)

# Create vessel
feed_n2 = Stream("N2 feed", n2)
feed_n2.setFlowRate(0.0, "kg/hr")
feed_n2.run()

vessel_n2 = VesselDepressurization("N2 Vessel", feed_n2)
vessel_n2.setVolume(1.0)
vessel_n2.setOrificeDiameter(0.01)
vessel_n2.setBackPressure(1.0)
vessel_n2.setCalculationType(VesselDepressurization.CalculationType.ISENTROPIC)
vessel_n2.run()

# Get gamma for ideal gas comparison
gamma = n2.getGamma()
T0 = 300.0
P0 = 50.0

# Run simulation
dt = 0.1
end_time = 60.0
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_n2.getPressure("bar")]
temp_data = [vessel_n2.getTemperature()]
temp_ideal = [T0]

for t in np.arange(dt, end_time + dt, dt):
    vessel_n2.runTransient(dt, uuid)
    time_data.append(t)
    P = vessel_n2.getPressure("bar")
    pressure_data.append(P)
    temp_data.append(vessel_n2.getTemperature())
    # Ideal isentropic: T/T0 = (P/P0)^((gamma-1)/gamma)
    temp_ideal.append(T0 * (P / P0)**((gamma - 1) / gamma))

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(time_data, pressure_data, 'b-', linewidth=2)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Pressure [bar]')
ax1.set_title('Isentropic N₂ Depressurization')
ax1.grid(True)

ax2.plot(time_data, temp_data, 'r-', linewidth=2, label='NeqSim (Real Gas)')
ax2.plot(time_data, temp_ideal, 'k--', linewidth=1.5, label='Ideal Gas Theory')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Temperature [K]')
ax2.set_title('Temperature Evolution')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"Gamma (Cp/Cv) = {gamma:.3f}")
print(f"Temperature drop: {T0:.0f} K → {temp_data[-1]:.0f} K = {T0 - temp_data[-1]:.0f} K")

## Example 3: Multi-Component Natural Gas with Energy Balance

NeqSim's advantage over HydDown: full multi-component mixture support with rigorous VLE.

In [ ]:
# Create natural gas mixture
ng = SystemSrkEos(300.0, 30.0)
ng.addComponent("methane", 0.85)
ng.addComponent("ethane", 0.08)
ng.addComponent("propane", 0.04)
ng.addComponent("n-butane", 0.02)
ng.addComponent("nitrogen", 0.01)
ng.setMixingRule("classic")
ng.createDatabase(True)

feed_ng = Stream("NG feed", ng)
feed_ng.setFlowRate(0.0, "kg/hr")
feed_ng.run()

vessel_ng = VesselDepressurization("NG Vessel", feed_ng)
vessel_ng.setVolume(1.0)
vessel_ng.setOrificeDiameter(0.008)  # 8mm
vessel_ng.setBackPressure(1.0)
vessel_ng.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel_ng.setHeatTransferType(VesselDepressurization.HeatTransferType.CALCULATED)
vessel_ng.setVesselGeometry(2.0, 0.8, VesselDepressurization.VesselOrientation.VERTICAL)
vessel_ng.setVesselProperties(0.015, 7800.0, 500.0, 45.0)  # Steel
vessel_ng.setAmbientTemperature(300.0)
vessel_ng.setExternalHeatTransferCoefficient(10.0)
vessel_ng.run()

# Run simulation
dt = 0.1
end_time = 120.0
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_ng.getPressure("bar")]
temp_data = [vessel_ng.getTemperature()]
wall_temp_data = [vessel_ng.getWallTemperature()]

for t in np.arange(dt, end_time + dt, dt):
    vessel_ng.runTransient(dt, uuid)
    time_data.append(t)
    pressure_data.append(vessel_ng.getPressure("bar"))
    temp_data.append(vessel_ng.getTemperature())
    wall_temp_data.append(vessel_ng.getWallTemperature())

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Pressure [bar]')
axes[0].set_title('Pressure')
axes[0].grid(True)

axes[1].plot(time_data, [t - 273.15 for t in temp_data], 'r-', linewidth=2, label='Gas')
axes[1].plot(time_data, [t - 273.15 for t in wall_temp_data], 'k--', linewidth=1.5, label='Wall')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Temperature [°C]')
axes[1].set_title('Temperature')
axes[1].legend()
axes[1].grid(True)

# Calculate mass flow rate from mass history
mass_hist = [vessel_ng.getMassHistory().get(i) for i in range(len(time_data))]
mdot = [0] + [(mass_hist[i-1] - mass_hist[i]) / dt for i in range(1, len(mass_hist))]

axes[2].plot(time_data, mdot, 'g-', linewidth=2)
axes[2].set_xlabel('Time [s]')
axes[2].set_ylabel('Mass flow [kg/s]')
axes[2].set_title('Discharge Rate')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print(f"Mixture MW = {ng.getMolarMass() * 1000:.2f} g/mol")
print(f"Final: P = {pressure_data[-1]:.1f} bar, T = {temp_data[-1] - 273.15:.1f} °C")

## Example 4: Type IV Hydrogen Vessel with 1-D Wall Conduction

Type IV vessels (HDPE liner + CFRP shell) have low thermal conductivity, requiring transient wall temperature modeling.

In [ ]:
# Create hydrogen system at 350 bar, 300 K (typical storage)
h2 = SystemSrkEos(300.0, 350.0)
h2.addComponent("hydrogen", 1.0)
h2.setMixingRule("classic")
h2.createDatabase(True)

feed_h2 = Stream("H2 feed", h2)
feed_h2.setFlowRate(0.0, "kg/hr")
feed_h2.run()

vessel_h2 = VesselDepressurization("H2 Type IV", feed_h2)
vessel_h2.setVesselGeometry(0.8, 0.23, VesselDepressurization.VesselOrientation.HORIZONTAL)

# Option 1: Manual properties (original approach)
# vessel_h2.setVesselProperties(0.017, 1360.0, 1020.0, 0.5)  # CFRP
# vessel_h2.setLinerProperties(0.007, 945.0, 1584.0, 0.385)  # HDPE

# Option 2: Use material presets (cleaner, less error-prone) - NEW API!
vessel_h2.setVesselMaterial(0.017, VesselDepressurization.VesselMaterial.CFRP)
vessel_h2.setLinerMaterial(0.007, VesselDepressurization.LinerMaterial.HDPE)

vessel_h2.setOrificeDiameter(0.003)  # 3mm (small for controlled discharge)
vessel_h2.setBackPressure(1.0)
vessel_h2.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel_h2.setHeatTransferType(VesselDepressurization.HeatTransferType.TRANSIENT_WALL)
vessel_h2.setAmbientTemperature(293.0)
vessel_h2.setExternalHeatTransferCoefficient(8.0)
vessel_h2.run()

print(f"Initial mass: {vessel_h2.getMass():.3f} kg H₂")
print(f"Volume: {vessel_h2.getVolume() * 1000:.1f} liters")

# Run simulation (slow discharge - 30 minutes)
dt = 1.0  # 1 second steps for faster simulation
end_time = 1800.0  # 30 minutes
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_h2.getPressure("bar")]
temp_data = [vessel_h2.getTemperature()]
wall_temp_data = [vessel_h2.getWallTemperature()]

print("\nSimulating 30-minute discharge...")
for t in np.arange(dt, end_time + dt, dt):
    vessel_h2.runTransient(dt, uuid)
    if t % 60 == 0:  # Record every minute
        time_data.append(t / 60)  # Convert to minutes
        pressure_data.append(vessel_h2.getPressure("bar"))
        temp_data.append(vessel_h2.getTemperature())
        wall_temp_data.append(vessel_h2.getWallTemperature())

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(time_data, pressure_data, 'b-', linewidth=2)
ax1.set_xlabel('Time [min]')
ax1.set_ylabel('Pressure [bar]')
ax1.set_title('H₂ Type IV Vessel Discharge')
ax1.grid(True)

ax2.plot(time_data, [t - 273.15 for t in temp_data], 'r-', linewidth=2, label='Gas')
ax2.plot(time_data, [t - 273.15 for t in wall_temp_data], 'k--', linewidth=1.5, label='Inner Wall')
ax2.axhline(y=293 - 273.15, color='gray', linestyle=':', label='Ambient')
ax2.set_xlabel('Time [min]')
ax2.set_ylabel('Temperature [°C]')
ax2.set_title('Temperature with 1-D Wall Conduction')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"\nFinal: P = {pressure_data[-1]:.1f} bar, T_gas = {temp_data[-1] - 273.15:.1f} °C")
print(f"Inner wall temp: {wall_temp_data[-1] - 273.15:.1f} °C")
print(f"Note: Low thermal conductivity of Type IV creates significant wall temperature lag")

## Example 5: Two-Phase CO₂ Depressurization

NEW feature: Separate wall temperatures for gas-wetted and liquid-wetted regions.

In [ ]:
# Two options for creating two-phase pure component systems:

# OPTION 1: Use the NEW helper method (recommended - one line!)
# co2 = VesselDepressurization.createTwoPhaseFluid("CO2", 250.0, 0.6)

# OPTION 2: Manual approach (shown for understanding)
from neqsim.thermodynamicoperations import ThermodynamicOperations

# First, calculate bubble point pressure at 250K
co2_init = SystemSrkEos(250.0, 1.0)  # Start with any pressure
co2_init.addComponent("CO2", 1.0)
co2_init.setMixingRule("classic")
co2_init.createDatabase(True)

# Calculate bubble point pressure at 250K
ops = ThermodynamicOperations(co2_init)
ops.bubblePointPressureFlash(False)

saturation_pressure = co2_init.getPressure()
print(f"CO2 bubble point pressure at 250K: {saturation_pressure:.2f} bar")

# Now create the two-phase system at saturation pressure
co2 = SystemSrkEos(250.0, saturation_pressure)
co2.addComponent("CO2", 1.0)
co2.setMixingRule("classic")
co2.createDatabase(True)

# Set up two-phase with desired vapor fraction
# beta = 0.0 → all liquid (bubble point)
# beta = 1.0 → all vapor (dew point)  
# beta = 0.6 → 60% vapor, 40% liquid by moles
co2.setNumberOfPhases(2)
co2.setBeta(0.6)
co2.init(0)
co2.init(1)

print(f"Set vapor fraction (beta): {co2.getBeta():.2f}")
print(f"At saturation: can have any beta from 0 (all liquid) to 1 (all vapor)")

feed_co2 = Stream("CO2 feed", co2)
feed_co2.setFlowRate(0.0, "kg/hr")
feed_co2.run()

print(f"\nNumber of phases in feed: {feed_co2.getThermoSystem().getNumberOfPhases()}")

vessel_co2 = VesselDepressurization("CO2 Vessel", feed_co2)
vessel_co2.setVesselGeometry(11.0, 2.0, VesselDepressurization.VesselOrientation.HORIZONTAL)

# Use material preset instead of manual properties - NEW API!
vessel_co2.setVesselMaterial(0.015, VesselDepressurization.VesselMaterial.CARBON_STEEL)

vessel_co2.setOrificeDiameter(0.02)  # 20mm (PSV-sized)
vessel_co2.setBackPressure(1.0)
vessel_co2.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel_co2.setHeatTransferType(VesselDepressurization.HeatTransferType.CALCULATED)

# Enable two-phase heat transfer modeling
vessel_co2.setTwoPhaseHeatTransfer(True)
vessel_co2.setInitialLiquidLevel(0.4)  # 40% filled with liquid

vessel_co2.setAmbientTemperature(300.0)
vessel_co2.setExternalHeatTransferCoefficient(10.0)
vessel_co2.run()

print(f"\nTwo-phase CO₂ vessel initialized")
print(f"Initial liquid level: 40%")
print(f"Vessel mass: {vessel_co2.getMass():.1f} kg")

# Run simulation
dt = 0.5
end_time = 300.0  # 5 minutes
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_co2.getPressure("bar")]
temp_data = [vessel_co2.getTemperature()]
gas_wall_data = [vessel_co2.getGasWallTemperature()]
liq_wall_data = [vessel_co2.getLiquidWallTemperature()]
liquid_level_data = [vessel_co2.getLiquidLevel()]

for t in np.arange(dt, end_time + dt, dt):
    vessel_co2.runTransient(dt, uuid)
    if t % 10 == 0:  # Record every 10 seconds
        time_data.append(t)
        pressure_data.append(vessel_co2.getPressure("bar"))
        temp_data.append(vessel_co2.getTemperature())
        gas_wall_data.append(vessel_co2.getGasWallTemperature())
        liq_wall_data.append(vessel_co2.getLiquidWallTemperature())
        liquid_level_data.append(vessel_co2.getLiquidLevel())

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Pressure [bar]')
axes[0].set_title('Two-Phase CO₂ Depressurization')
axes[0].grid(True)

axes[1].plot(time_data, [t - 273.15 for t in temp_data], 'b-', linewidth=2, label='Fluid')
axes[1].plot(time_data, [t - 273.15 for t in gas_wall_data], 'r--', linewidth=1.5, label='Gas Wall')
axes[1].plot(time_data, [t - 273.15 for t in liq_wall_data], 'g--', linewidth=1.5, label='Liquid Wall')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Temperature [°C]')
axes[1].set_title('Separate Wall Temperatures')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(time_data, [l * 100 for l in liquid_level_data], 'm-', linewidth=2)
axes[2].set_xlabel('Time [s]')
axes[2].set_ylabel('Liquid Level [%]')
axes[2].set_title('Liquid Level Evolution')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print(f"\nFinal gas wall temp: {gas_wall_data[-1] - 273.15:.1f} °C")
print(f"Final liquid wall temp: {liq_wall_data[-1] - 273.15:.1f} °C")

## Example 5b: Multi-Component Two-Phase NGL Depressurization

This example demonstrates two-phase depressurization of a multi-component NGL (Natural Gas Liquids) mixture with separate gas and liquid wall temperature tracking. The mixture contains light and heavy hydrocarbons that separate into vapor and liquid phases during blowdown.

In [ ]:
# Create multi-component NGL mixture (typical LPG/condensate)
# This will form two phases: lighter components in vapor, heavier in liquid
ngl = SystemSrkEos(280.0, 20.0)  # 7°C, 20 bara - conditions for two-phase
ngl.addComponent("methane", 0.15)
ngl.addComponent("ethane", 0.25)
ngl.addComponent("propane", 0.30)
ngl.addComponent("n-butane", 0.15)
ngl.addComponent("n-pentane", 0.10)
ngl.addComponent("n-hexane", 0.05)
ngl.setMixingRule("classic")
ngl.createDatabase(True)
ngl.setMultiPhaseCheck(True)

feed_ngl = Stream("NGL feed", ngl)
feed_ngl.setFlowRate(0.0, "kg/hr")  # Closed vessel
feed_ngl.run()

print("=== Multi-Component NGL Two-Phase System ===")
print(f"Components: CH4, C2H6, C3H8, nC4, nC5, nC6")
print(f"Initial conditions: {feed_ngl.getTemperature() - 273.15:.1f} °C, {feed_ngl.getPressure():.1f} bara")
print(f"Number of phases: {feed_ngl.getThermoSystem().getNumberOfPhases()}")

# Show phase compositions
fluid = feed_ngl.getThermoSystem()
if fluid.getNumberOfPhases() > 1:
    print("\nVapor phase molar composition:")
    for i in range(fluid.getNumberOfComponents()):
        print(f"  {fluid.getComponent(i).getName()}: {fluid.getPhase('gas').getComponent(i).getx():.3f}")
    print("\nLiquid phase molar composition:")
    for i in range(fluid.getNumberOfComponents()):
        print(f"  {fluid.getComponent(i).getName()}: {fluid.getPhase('oil').getComponent(i).getx():.3f}")

# Configure vessel
vessel_ngl = VesselDepressurization("NGL Vessel", feed_ngl)
vessel_ngl.setVesselGeometry(8.0, 2.5, VesselDepressurization.VesselOrientation.HORIZONTAL)
vessel_ngl.setVesselProperties(0.020, 7800.0, 490.0, 45.0)  # 20mm steel wall
vessel_ngl.setOrificeDiameter(0.025)  # 25mm orifice (blowdown valve)
vessel_ngl.setBackPressure(1.0)
vessel_ngl.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel_ngl.setHeatTransferType(VesselDepressurization.HeatTransferType.CALCULATED)

# Enable two-phase heat transfer with separate wall zones
vessel_ngl.setTwoPhaseHeatTransfer(True)
vessel_ngl.setInitialLiquidLevel(0.50)  # 50% liquid level

vessel_ngl.setAmbientTemperature(288.15)  # 15°C ambient
vessel_ngl.setExternalHeatTransferCoefficient(10.0)  # Natural convection outside
vessel_ngl.run()

print(f"\n=== Vessel Configuration ===")
print(f"Volume: {vessel_ngl.getVolume():.1f} m³")
print(f"Initial liquid level: 50%")
print(f"Initial vapor fraction: {vessel_ngl.getVaporFraction():.3f}")

# Run transient simulation
dt = 0.5
end_time = 600.0  # 10 minutes blowdown
uuid = UUID.randomUUID()

# Data storage
time_data = [0]
pressure_data = [vessel_ngl.getPressure("bar")]
temp_data = [vessel_ngl.getTemperature()]
gas_wall_data = [vessel_ngl.getGasWallTemperature()]
liq_wall_data = [vessel_ngl.getLiquidWallTemperature()]
liquid_level_data = [vessel_ngl.getLiquidLevel()]
vapor_frac_data = [vessel_ngl.getVaporFraction()]
mass_data = [vessel_ngl.getFluidMass()]

print(f"\nRunning 10-minute blowdown simulation...")

for t in np.arange(dt, end_time + dt, dt):
    vessel_ngl.runTransient(dt, uuid)
    if t % 20 == 0:  # Record every 20 seconds
        time_data.append(t)
        pressure_data.append(vessel_ngl.getPressure("bar"))
        temp_data.append(vessel_ngl.getTemperature())
        gas_wall_data.append(vessel_ngl.getGasWallTemperature())
        liq_wall_data.append(vessel_ngl.getLiquidWallTemperature())
        liquid_level_data.append(vessel_ngl.getLiquidLevel())
        vapor_frac_data.append(vessel_ngl.getVaporFraction())
        mass_data.append(vessel_ngl.getFluidMass())

# Plot comprehensive results
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Pressure vs time
axes[0, 0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Pressure [bara]')
axes[0, 0].set_title('NGL Vessel Pressure')
axes[0, 0].grid(True, alpha=0.3)

# Temperature vs time - fluid and both wall zones
axes[0, 1].plot(time_data, [t - 273.15 for t in temp_data], 'b-', linewidth=2, label='Fluid (bulk)')
axes[0, 1].plot(time_data, [t - 273.15 for t in gas_wall_data], 'r--', linewidth=1.5, label='Gas-wetted wall')
axes[0, 1].plot(time_data, [t - 273.15 for t in liq_wall_data], 'g--', linewidth=1.5, label='Liquid-wetted wall')
axes[0, 1].axhline(y=288.15 - 273.15, color='orange', linestyle=':', label='Ambient (15°C)')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Temperature [°C]')
axes[0, 1].set_title('Temperatures During Blowdown')
axes[0, 1].legend(fontsize=8)
axes[0, 1].grid(True, alpha=0.3)

# Liquid level
axes[0, 2].plot(time_data, [l * 100 for l in liquid_level_data], 'm-', linewidth=2)
axes[0, 2].set_xlabel('Time [s]')
axes[0, 2].set_ylabel('Liquid Level [%]')
axes[0, 2].set_title('Liquid Level Evolution')
axes[0, 2].set_ylim([0, 100])
axes[0, 2].grid(True, alpha=0.3)

# Vapor fraction
axes[1, 0].plot(time_data, vapor_frac_data, 'c-', linewidth=2)
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Vapor Fraction [-]')
axes[1, 0].set_title('Vapor Fraction (molar)')
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(True, alpha=0.3)

# Mass remaining
axes[1, 1].plot(time_data, mass_data, 'k-', linewidth=2)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Mass [kg]')
axes[1, 1].set_title('Remaining Fluid Mass')
axes[1, 1].grid(True, alpha=0.3)

# Wall temperature difference (shows asymmetry of heat transfer)
wall_diff = [g - l for g, l in zip(gas_wall_data, liq_wall_data)]
axes[1, 2].plot(time_data, wall_diff, 'purple', linewidth=2)
axes[1, 2].axhline(y=0, color='gray', linestyle='-', alpha=0.5)
axes[1, 2].set_xlabel('Time [s]')
axes[1, 2].set_ylabel('ΔT (Gas Wall - Liquid Wall) [K]')
axes[1, 2].set_title('Wall Temperature Asymmetry')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print(f"\n=== Blowdown Summary ===")
print(f"Duration: {end_time:.0f} seconds ({end_time/60:.1f} minutes)")
print(f"Pressure: {pressure_data[0]:.1f} → {pressure_data[-1]:.1f} bara")
print(f"Fluid temperature: {temp_data[0] - 273.15:.1f} → {temp_data[-1] - 273.15:.1f} °C")
print(f"Gas wall temperature: {gas_wall_data[0] - 273.15:.1f} → {gas_wall_data[-1] - 273.15:.1f} °C")
print(f"Liquid wall temperature: {liq_wall_data[0] - 273.15:.1f} → {liq_wall_data[-1] - 273.15:.1f} °C")
print(f"Liquid level: {liquid_level_data[0]*100:.1f}% → {liquid_level_data[-1]*100:.1f}%")
print(f"Mass released: {mass_data[0] - mass_data[-1]:.1f} kg ({100*(1-mass_data[-1]/mass_data[0]):.1f}%)")
print(f"\nNote: Liquid wall temperature drops faster due to:")
print(f"  - Higher heat transfer coefficient (boiling/evaporation)")
print(f"  - Direct contact with cold evaporating liquid")
print(f"  - This is critical for MDMT (Minimum Design Metal Temperature) assessment")

## Example 6: API 521 PSV Sizing Comparison

Compare dynamic simulation against API 521 steady-state relief valve sizing.

In [ ]:
# Fire case: propane vessel at set pressure
propane = SystemSrkEos(300.0, 17.0)  # 17 bara set pressure
propane.addComponent("propane", 1.0)
propane.setMixingRule("classic")
propane.createDatabase(True)

# Use ReliefValveSizing for API 521 calculation
psv = ReliefValveSizing(propane)
psv.setVesselVolume(10.0)  # 10 m³
psv.setSetPressure(17.0)  # bara
psv.setOverpressure(0.21)  # 21% overpressure
psv.setBackPressure(1.0)  # bara

# Fire heat input per API 521 (wetted area)
wetted_area = 20.0  # m² (example)
F_factor = 1.0  # No insulation
Q_fire = ReliefValveSizing.calculateAPI521HeatInput(wetted_area, F_factor)

psv.setHeatInput(Q_fire)

# Calculate required orifice area
result = psv.calculateRequiredArea()

print("API 521 Relief Valve Sizing (Fire Case)")
print("="*45)
print(f"Vessel volume: 10.0 m³")
print(f"Set pressure: 17.0 bara")
print(f"Relieving pressure: {result.getRelievingPressure():.1f} bara")
print(f"Fire heat input: {Q_fire/1000:.0f} kW")
print(f"Required mass flow: {result.getMassFlowRate():.2f} kg/s")
print(f"Required orifice area: {result.getRequiredArea() * 1e6:.1f} mm²")
print(f"Recommended API orifice: {result.getRecommendedOrifice()}")

## Summary

This notebook demonstrated NeqSim's vessel depressurization capabilities:

- **Calculation types**: Isothermal, Isentropic, Isenthalpic, Isenergetic, Energy Balance
- **Multi-component mixtures**: Full VLE flash calculations for any number of components
- **Heat transfer modeling**: Natural/mixed convection with 1-D wall conduction
- **Two-phase support**: Separate gas and liquid wall temperature tracking
- **API 521 PSV sizing**: Dynamic relief valve sizing for fire scenarios

### Key Classes
- `VesselDepressurization` - Main dynamic vessel model
- `VesselHeatTransferCalculator` - Natural/mixed convection correlations
- `TransientWallHeatTransfer` - 1-D wall conduction solver
- `ReliefValveSizing` - API 521 PSV sizing

### NEW: Improved API (v2.0)

The following convenience features simplify blowdown simulation design:

| Feature | Description |
|---------|-------------|
| **Material Presets** | `VesselMaterial.CARBON_STEEL`, `VesselMaterial.STAINLESS_304`, `VesselMaterial.CFRP`, etc. |
| **Liner Presets** | `LinerMaterial.HDPE`, `LinerMaterial.NYLON`, `LinerMaterial.ALUMINUM` |
| **`runSimulation()`** | Runs complete simulation and returns structured `SimulationResult` object |
| **`validate()`** | Checks configuration for common errors before running |
| **`createTwoPhaseFluid()`** | Static helper to create two-phase pure component systems |

## Example 7: Improved API Demo

The new API provides material presets, a convenient `runSimulation()` method, and validation.

In [ ]:
# ============================================================
# Example 7a: Using Material Presets (cleaner, less error-prone)
# ============================================================

# OLD WAY (manual properties - easy to make mistakes):
# vessel.setVesselProperties(0.015, 7800.0, 500.0, 45.0)  # What are these numbers?

# NEW WAY (clear intent, validated properties):
gas = SystemSrkEos(300.0, 100.0)
gas.addComponent("hydrogen", 1.0)
gas.setMixingRule("classic")
gas.createDatabase(True)

feed = Stream("H2 feed", gas)
feed.setFlowRate(0.0, "kg/hr")
feed.run()

vessel = VesselDepressurization("H2 Type IV Tank", feed)
vessel.setVesselGeometry(0.8, 0.23, VesselDepressurization.VesselOrientation.HORIZONTAL)

# Use material presets - self-documenting and validated properties!
vessel.setVesselMaterial(0.017, VesselDepressurization.VesselMaterial.CFRP)
vessel.setLinerMaterial(0.007, VesselDepressurization.LinerMaterial.HDPE)

vessel.setOrificeDiameter(0.003)
vessel.setBackPressure(1.0)
vessel.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel.setHeatTransferType(VesselDepressurization.HeatTransferType.TRANSIENT_WALL)
vessel.setAmbientTemperature(293.0)
vessel.setExternalHeatTransferCoefficient(8.0)
vessel.run()

print("=== Material Presets Demo ===")
print("Available materials:")
for mat in VesselDepressurization.VesselMaterial.values():
    print(f"  {mat.name()}: ρ={mat.getDensity():.0f} kg/m³, Cp={mat.getHeatCapacity():.0f} J/(kg·K), k={mat.getThermalConductivity():.1f} W/(m·K)")

print("\nVessel configured with CFRP shell + HDPE liner")
print(f"Initial pressure: {vessel.getPressure('bar'):.1f} bar")

In [ ]:
# ============================================================
# Example 7b: Using runSimulation() - No manual loop needed!
# ============================================================

# OLD WAY (manual time loop):
# time_data = [0]; pressure_data = [vessel.getPressure()]
# for t in np.arange(dt, end_time + dt, dt):
#     vessel.runTransient(dt, uuid)
#     time_data.append(t)
#     pressure_data.append(vessel.getPressure())

# NEW WAY (one line, structured result):
result = vessel.runSimulation(300.0, 0.5, 10)  # 5 minutes, 0.5s steps, record every 10th

print("\n=== runSimulation() Result Object ===")
print(f"Data points: {result.size()}")
print(f"Simulation: {result.getEndTime()}s with {result.getTimeStep()}s steps")
print(f"Pressure: {result.getInitialPressure():.1f} → {result.getFinalPressure():.1f} bar")
print(f"Temperature: {result.getInitialTemperature() - 273.15:.1f} → {result.getFinalTemperature() - 273.15:.1f} °C")
print(f"Mass discharged: {result.getMassDischarged():.3f} kg ({100*result.getMassDischargedFraction():.1f}%)")
print(f"Minimum temperature reached: {result.getMinTemperature() - 273.15:.1f} °C")

# Easy plotting from result object
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(result.getTime(), result.getPressure(), 'b-', linewidth=2)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Pressure [bar]')
ax1.set_title('Pressure (from SimulationResult)')
ax1.grid(True)

ax2.plot(result.getTime(), [t - 273.15 for t in result.getTemperature()], 'r-', linewidth=2, label='Gas')
ax2.plot(result.getTime(), [t - 273.15 for t in result.getWallTemperature()], 'k--', linewidth=1.5, label='Wall')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Temperature [°C]')
ax2.set_title('Temperature (from SimulationResult)')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Example 7c: Validation - Catch errors before running
# ============================================================

# Create a vessel with intentional configuration issues
gas_bad = SystemSrkEos(300.0, 50.0)
gas_bad.addComponent("methane", 1.0)
gas_bad.setMixingRule("classic")
gas_bad.createDatabase(True)

feed_bad = Stream("bad feed", gas_bad)
feed_bad.run()

vessel_bad = VesselDepressurization("BadConfig", feed_bad)
vessel_bad.setVolume(1.0)
vessel_bad.setOrificeDiameter(0.01)
vessel_bad.setBackPressure(60.0)  # ERROR: Higher than initial pressure!
vessel_bad.setTwoPhaseHeatTransfer(True)  # WARNING: No liquid level set
vessel_bad.run()

print("=== Validation Demo ===")
print("\nChecking for warnings (non-blocking):")
warnings = vessel_bad.validateWithWarnings()
for w in warnings:
    print(f"  ⚠️  {w}")

print("\nValidation catches configuration errors before wasting compute time!")
print("Call vessel.validate() to throw exception on errors, or")
print("vessel.validateWithWarnings() to get a list of issues.")

In [ ]:
# ============================================================
# Example 7d: Two-Phase Fluid Helper - Simplified pure component setup
# ============================================================

# OLD WAY (verbose, error-prone):
# co2_init = SystemSrkEos(250.0, 1.0)
# co2_init.addComponent("CO2", 1.0)
# ...
# ops = ThermodynamicOperations(co2_init)
# ops.bubblePointPressureFlash(False)
# saturation_pressure = co2_init.getPressure()
# co2 = SystemSrkEos(250.0, saturation_pressure)
# ...

# NEW WAY (one line!):
co2_easy = VesselDepressurization.createTwoPhaseFluid("CO2", 250.0, 0.6)  # 250K, 60% vapor

print("=== createTwoPhaseFluid() Helper ===")
print(f"Created CO2 at T={co2_easy.getTemperature():.1f}K, P={co2_easy.getPressure():.2f} bar")
print(f"Number of phases: {co2_easy.getNumberOfPhases()}")
print(f"Vapor fraction (beta): {co2_easy.getBeta():.2f}")
print("\nThis helper automatically:")
print("  1. Calculates bubble point pressure at the given temperature")
print("  2. Creates system at saturation conditions")
print("  3. Sets up two phases with specified vapor fraction")

# Can also create by specifying pressure instead of temperature
propane_easy = VesselDepressurization.createTwoPhaseFluidAtPressure("propane", 10.0, 0.5)
print(f"\nPropane at 10 bar: T_sat = {propane_easy.getTemperature() - 273.15:.1f}°C")

## Example 8: Flare System Integration

VesselDepressurization integrates with NeqSim's process equipment for complete blowdown system modeling.

**Key Integration Points:**
- `getOutletStream()` - Connect to Flare or flare header (Mixer)
- `getDischargeRate()` / `getPeakDischargeRate()` - For flare sizing
- `getTimeToReachPressure()` - API 521 blowdown time verification
- `getMinimumWallTemperatureReached()` - MDMT assessment

In [ ]:
# ============================================================
# Example 8a: Connect Vessel Blowdown to Flare
# ============================================================

from neqsim.process.equipment.flare import Flare
from neqsim.process.equipment.mixer import Mixer

# Create a propane vessel for ESD blowdown scenario
propane = SystemSrkEos(300.0, 20.0)  # 20 bara, 27°C
propane.addComponent("propane", 1.0)
propane.setMixingRule("classic")
propane.createDatabase(True)

feed_propane = Stream("Propane feed", propane)
feed_propane.setFlowRate(0.0, "kg/hr")  # Closed vessel
feed_propane.run()

# Create vessel with ESD-sized blowdown orifice
vessel_esd = VesselDepressurization("Propane Storage", feed_propane)
vessel_esd.setVesselGeometry(10.0, 2.5, VesselDepressurization.VesselOrientation.HORIZONTAL)
vessel_esd.setVesselMaterial(0.020, VesselDepressurization.VesselMaterial.CARBON_STEEL)
vessel_esd.setOrificeDiameter(0.025)  # 25mm blowdown orifice
vessel_esd.setBackPressure(1.5)  # Flare header pressure
vessel_esd.setCalculationType(VesselDepressurization.CalculationType.ENERGY_BALANCE)
vessel_esd.setHeatTransferType(VesselDepressurization.HeatTransferType.CALCULATED)
vessel_esd.setAmbientTemperature(300.0)
vessel_esd.setExternalHeatTransferCoefficient(10.0)
vessel_esd.run()

print("=== ESD Blowdown to Flare System ===")
print(f"Vessel: {vessel_esd.getVolume():.1f} m³, {vessel_esd.getMass():.0f} kg propane")
print(f"Initial pressure: {vessel_esd.getPressure('bar'):.1f} bar")

# Connect outlet stream to flare header
flare_header = Mixer("Flare Header")
flare_header.addStream(vessel_esd.getOutletStream())

# Create emergency flare
flare = Flare("Emergency Flare", flare_header.getOutletStream())
flare.setFlameHeight(50.0)  # 50m flame for blowdown event
flare.setRadiantFraction(0.20)
flare.setTipDiameter(0.8)

print("\nFlare system connected!")
print(f"Flare tip diameter: 0.8 m")
print(f"Flame height: 50 m")

In [ ]:
# ============================================================
# Example 8b: Run Blowdown with Flare Tracking
# ============================================================

# Run blowdown simulation
dt = 0.5
end_time = 600.0  # 10 minutes
uuid = UUID.randomUUID()

# Data collection
time_data = [0]
pressure_data = [vessel_esd.getPressure("bar")]
discharge_rate_data = [0]
flare_heat_data = [0]
flare_co2_data = [0]
cumulative_co2 = [0]

# Reset flare cumulative counters
flare.resetCumulative()

print("Running blowdown simulation with flare tracking...")

for t in np.arange(dt, end_time + dt, dt):
    # Run vessel transient
    vessel_esd.runTransient(dt, uuid)
    
    # Update flare header and flare
    flare_header.run()
    flare.run()
    flare.updateCumulative(dt)  # Track cumulative emissions
    
    if t % 30 == 0:  # Record every 30 seconds
        time_data.append(t)
        pressure_data.append(vessel_esd.getPressure("bar"))
        discharge_rate_data.append(vessel_esd.getDischargeRate("kg/s"))
        flare_heat_data.append(flare.getHeatDuty("MW"))
        flare_co2_data.append(flare.getCO2Emission("kg/hr"))
        cumulative_co2.append(flare.getCumulativeCO2Emission("kg"))

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pressure profile
axes[0, 0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0, 0].axhline(y=7.0, color='r', linestyle='--', label='50% pressure (API 521 target)')
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Pressure [bar]')
axes[0, 0].set_title('Vessel Depressurization')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Mass discharge rate
axes[0, 1].plot(time_data, discharge_rate_data, 'g-', linewidth=2)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Discharge Rate [kg/s]')
axes[0, 1].set_title('Blowdown Mass Flow to Flare')
axes[0, 1].grid(True, alpha=0.3)

# Flare heat duty
axes[1, 0].plot(time_data, flare_heat_data, 'orange', linewidth=2)
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Flare Heat Release [MW]')
axes[1, 0].set_title('Flare Heat Duty')
axes[1, 0].grid(True, alpha=0.3)

# Cumulative CO2 emissions
axes[1, 1].plot(time_data, cumulative_co2, 'purple', linewidth=2)
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Cumulative CO₂ [kg]')
axes[1, 1].set_title('CO₂ Emissions')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary metrics
print("\n=== Blowdown Summary ===")
print(f"Blowdown time: {end_time/60:.1f} minutes")
print(f"Final pressure: {pressure_data[-1]:.2f} bar")
print(f"Time to 50% pressure (7 bar): {vessel_esd.getTimeToReachPressure(7.0):.0f} s")
print(f"\n=== Flare Performance ===")
print(f"Peak discharge rate: {vessel_esd.getPeakDischargeRate('kg/s'):.2f} kg/s ({vessel_esd.getPeakDischargeRate('kg/hr'):.0f} kg/hr)")
print(f"Peak flare heat duty: {max(flare_heat_data):.2f} MW")
print(f"Total mass to flare: {vessel_esd.getTotalMassDischarged('kg'):.0f} kg")
print(f"Total CO₂ emitted: {flare.getCumulativeCO2Emission('kg'):.0f} kg ({flare.getCumulativeCO2Emission('tonnes'):.2f} tonnes)")
print(f"\n=== MDMT Assessment ===")
print(f"Minimum fluid temperature: {vessel_esd.getMinimumTemperatureReached('C'):.1f} °C")
print(f"Minimum wall temperature: {vessel_esd.getMinimumWallTemperatureReached('C'):.1f} °C")

### Example 8c: Multiple Vessels to Common Flare Header

In real facilities, multiple vessels may blow down simultaneously to a common flare header. This is the design case for flare system sizing per API 521.

In [ ]:
# ============================================================
# Example 8c: Parallel Blowdown from Multiple Vessels
# ============================================================

# Define a factory function to create configured vessels
def create_blowdown_vessel(name, volume_m3, pressure_bar, temp_C, orifice_mm, gas_type):
    """Create a configured blowdown vessel."""
    from neqsim.thermo import ThermodynamicOperations
    
    fluid = SystemSrkEos(temp_C + 273.15, pressure_bar)
    if gas_type == "methane":
        fluid.addComponent("methane", 0.9)
        fluid.addComponent("ethane", 0.07)
        fluid.addComponent("propane", 0.03)
    elif gas_type == "hydrogen":
        fluid.addComponent("hydrogen", 0.95)
        fluid.addComponent("nitrogen", 0.05)
    else:  # nitrogen
        fluid.addComponent("nitrogen", 1.0)
    
    fluid.setMixingRule("classic")
    ThermodynamicOperations(fluid).TPflash()
    
    vessel = VesselDepressurization(name, fluid)
    vessel.setVolume(volume_m3, "m3")
    vessel.setOrificeType(2)  # Orifice
    vessel.setOrificeDiameter(orifice_mm / 1000.0)  # Convert mm to m
    vessel.setCalcType(5)  # Transient with heat transfer
    vessel.setVesselMaterial(VesselMaterial.CARBON_STEEL)
    vessel.setWallThickness(20 / 1000.0)  # 20mm wall
    vessel.run()
    return vessel

# Create three vessels with different conditions (API 521 fire scenario)
print("Creating multiple vessels for parallel blowdown...")
vessel_hp = create_blowdown_vessel("HP Separator", 
                                   volume_m3=15.0, 
                                   pressure_bar=60.0, 
                                   temp_C=50.0, 
                                   orifice_mm=25.0, 
                                   gas_type="methane")

vessel_mp = create_blowdown_vessel("MP Separator", 
                                   volume_m3=25.0, 
                                   pressure_bar=25.0, 
                                   temp_C=40.0, 
                                   orifice_mm=30.0, 
                                   gas_type="methane")

vessel_h2 = create_blowdown_vessel("H2 Buffer Tank", 
                                   volume_m3=5.0, 
                                   pressure_bar=100.0, 
                                   temp_C=30.0, 
                                   orifice_mm=15.0, 
                                   gas_type="hydrogen")

# Connect all to common flare header
common_header = Mixer("Common Flare Header")
common_header.addStream(vessel_hp.getOutletStream())
common_header.addStream(vessel_mp.getOutletStream())
common_header.addStream(vessel_h2.getOutletStream())

main_flare = Flare("Main Flare", common_header.getOutletStream())
main_flare.setFlameHeight(30.0)  # 30m flame for high load

# Run parallel blowdown simulation
dt = 1.0
end_time = 900.0  # 15 minutes

time_data = [0]
total_flow = [0]
hp_flow = [0]
mp_flow = [0]
h2_flow = [0]
flare_heat = [0]

main_flare.resetCumulative()
uuid_hp = UUID.randomUUID()
uuid_mp = UUID.randomUUID()
uuid_h2 = UUID.randomUUID()

print("Running parallel blowdown from 3 vessels...")

for t in np.arange(dt, end_time + dt, dt):
    # Run all vessels
    vessel_hp.runTransient(dt, uuid_hp)
    vessel_mp.runTransient(dt, uuid_mp)
    vessel_h2.runTransient(dt, uuid_h2)
    
    # Update header and flare
    common_header.run()
    main_flare.run()
    main_flare.updateCumulative(dt)
    
    if t % 15 == 0:  # Record every 15 seconds
        time_data.append(t)
        hp_flow.append(vessel_hp.getDischargeRate("kg/s"))
        mp_flow.append(vessel_mp.getDischargeRate("kg/s"))
        h2_flow.append(vessel_h2.getDischargeRate("kg/s"))
        total_flow.append(hp_flow[-1] + mp_flow[-1] + h2_flow[-1])
        flare_heat.append(main_flare.getHeatDuty("MW"))

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual and total flow rates
axes[0].plot(time_data, hp_flow, 'b-', linewidth=2, label=f'HP Separator ({vessel_hp.getPeakDischargeRate("kg/s"):.1f} kg/s peak)')
axes[0].plot(time_data, mp_flow, 'g-', linewidth=2, label=f'MP Separator ({vessel_mp.getPeakDischargeRate("kg/s"):.1f} kg/s peak)')
axes[0].plot(time_data, h2_flow, 'r-', linewidth=2, label=f'H2 Tank ({vessel_h2.getPeakDischargeRate("kg/s"):.1f} kg/s peak)')
axes[0].plot(time_data, total_flow, 'k--', linewidth=3, label=f'Total to Flare')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Mass Flow [kg/s]')
axes[0].set_title('Parallel Blowdown Flow Rates')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Combined flare heat duty
axes[1].fill_between(time_data, 0, flare_heat, alpha=0.5, color='orange')
axes[1].plot(time_data, flare_heat, 'r-', linewidth=2)
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Flare Heat Duty [MW]')
axes[1].set_title('Combined Flare Load')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Sizing summary
print("\n" + "="*60)
print("FLARE SYSTEM SIZING SUMMARY (API 521 Fire Case)")
print("="*60)
print(f"\n{'Vessel':<20} {'Volume':<12} {'Init P':<10} {'Orifice':<10} {'Peak Flow':<15}")
print(f"{'':<20} {'[m³]':<12} {'[bar]':<10} {'[mm]':<10} {'[kg/s]':<15}")
print("-"*60)
print(f"{'HP Separator':<20} {'15.0':<12} {'60.0':<10} {'25.0':<10} {vessel_hp.getPeakDischargeRate('kg/s'):<15.2f}")
print(f"{'MP Separator':<20} {'25.0':<12} {'25.0':<10} {'30.0':<10} {vessel_mp.getPeakDischargeRate('kg/s'):<15.2f}")
print(f"{'H2 Buffer Tank':<20} {'5.0':<12} {'100.0':<10} {'15.0':<10} {vessel_h2.getPeakDischargeRate('kg/s'):<15.2f}")
print("-"*60)
combined_peak = max(total_flow)
print(f"{'COMBINED PEAK':<20} {'':<12} {'':<10} {'':<10} {combined_peak:<15.2f}")
print(f"\nPeak flare heat duty: {max(flare_heat):.2f} MW")
print(f"Total CO₂ emitted: {main_flare.getCumulativeCO2Emission('tonnes'):.2f} tonnes")
print(f"\nNote: Coincident peak < sum of individual peaks due to staggered blowdown curves")

### Example 8d: Integration with Process Safety Equipment

The `VesselDepressurization` class provides key parameters needed for sizing other safety equipment:
- **Relief Valve Sizing**: Peak discharge rate and fluid properties at relieving conditions
- **Flare Header Sizing**: Peak combined flow, fluid density, and velocity
- **MDMT Assessment**: Minimum temperatures reached during blowdown

In [ ]:
# ============================================================
# Example 8d: Safety Equipment Sizing Summary
# ============================================================

# Use the HP Separator from previous example for sizing demonstration
vessel = vessel_hp

print("="*70)
print("PROCESS SAFETY SIZING DATA FROM BLOWDOWN SIMULATION")
print("="*70)

# Section 1: Blowdown Valve Sizing (BDV / BV)
print("\n1. BLOWDOWN VALVE SIZING (API 520/521)")
print("-" * 50)
print(f"   Required capacity: {vessel.getPeakDischargeRate('kg/hr'):.0f} kg/hr")
print(f"                      ({vessel.getPeakDischargeRate('kg/s'):.2f} kg/s)")
print(f"   Inlet conditions at peak flow:")
print(f"     - Pressure: {vessel.getPressure('bar'):.1f} bar (current)")
print(f"     - Temperature: {vessel.getTemperature('C'):.1f} °C")

# Section 2: Flare Header Sizing
print("\n2. FLARE HEADER SIZING")
print("-" * 50)
outlet_stream = vessel.getOutletStream()
gas = outlet_stream.getFluid()
if gas.hasPhaseType("gas"):
    gas_density = gas.getPhase("gas").getDensity("kg/m3")
    print(f"   Gas density at outlet: {gas_density:.2f} kg/m³")
    
    # Calculate volumetric flow
    vol_flow = vessel.getPeakDischargeRate("kg/s") / gas_density if gas_density > 0 else 0
    print(f"   Peak volumetric flow: {vol_flow:.2f} m³/s")
    
    # Header sizing (typically 30-60 m/s velocity in flare headers)
    for velocity in [30, 45, 60]:
        area = vol_flow / velocity if velocity > 0 else 0
        diameter = (4 * area / 3.14159) ** 0.5 if area > 0 else 0
        print(f"   At {velocity} m/s → Header diameter: {diameter*1000:.0f} mm ({diameter*1000/25.4:.1f} inch)")

# Section 3: MDMT Assessment
print("\n3. MINIMUM DESIGN METAL TEMPERATURE (MDMT)")
print("-" * 50)
min_fluid_temp = vessel.getMinimumTemperatureReached("C")
min_wall_temp = vessel.getMinimumWallTemperatureReached("C")
print(f"   Minimum fluid temperature: {min_fluid_temp:.1f} °C")
print(f"   Minimum wall temperature: {min_wall_temp:.1f} °C")
print(f"\n   MDMT recommendations:")
if min_wall_temp < -29:
    print(f"   ⚠️  Very low temperatures - consider impact testing per ASME B31.3")
    print(f"   ⚠️  Carbon steel may not be suitable - evaluate 304SS or special CS grade")
elif min_wall_temp < 0:
    print(f"   ⚠️  Sub-zero temperatures - verify CS impact properties at {min_wall_temp:.0f}°C")
else:
    print(f"   ✓  Wall temperature stays above 0°C - standard CS acceptable")

# Section 4: API 521 Blowdown Time
print("\n4. API 521 BLOWDOWN TIME COMPLIANCE")
print("-" * 50)
initial_p = 60.0  # bar - this was the initial pressure
target_p = initial_p * 0.5  # 50% depressurization per API 521
blowdown_time = vessel.getTimeToReachPressure(target_p)
print(f"   Initial pressure: {initial_p:.0f} bar")
print(f"   Target pressure (50%): {target_p:.0f} bar")
print(f"   Time to reach target: {blowdown_time:.0f} seconds")
print(f"\n   API 521 requirement: Typically 15 minutes for fire case")
if blowdown_time <= 900:
    print(f"   ✓  COMPLIANT - Blowdown completes in {blowdown_time/60:.1f} minutes")
else:
    print(f"   ⚠️  NON-COMPLIANT - Blowdown takes {blowdown_time/60:.1f} minutes")
    print(f"       Consider increasing orifice size")

# Section 5: Summary for Design Datasheet
print("\n" + "="*70)
print("SUMMARY FOR DESIGN DATASHEET")
print("="*70)
print(f"""
Vessel:               {vessel.getName()}
Volume:               {vessel.getVolume('m3'):.1f} m³
Initial Pressure:     {initial_p:.0f} barg
Orifice Size:         {vessel.getOrificeDiameter()*1000:.0f} mm

BLOWDOWN PERFORMANCE:
- Peak discharge:     {vessel.getPeakDischargeRate('kg/hr'):.0f} kg/hr
- Time to 50%:        {blowdown_time:.0f} s ({blowdown_time/60:.1f} min)
- Total mass vented:  {vessel.getTotalMassDischarged('kg'):.0f} kg

TEMPERATURE LIMITS:
- Min fluid temp:     {min_fluid_temp:.1f} °C
- Min wall temp:      {min_wall_temp:.1f} °C
- Wall material MDMT: {min_wall_temp - 10:.0f} °C (with 10°C margin)
""")

## Example 9: Advanced Features

This example demonstrates the newly added advanced features:
1. **Fire Case (API 521)**: External fire heat input during depressurization
2. **Valve Opening Dynamics**: ESD valve opening time modeling
3. **Data Export**: CSV and JSON export for post-processing
4. **Liquid Rainout Detection**: Two-phase flow analysis for flare headers
5. **Flare Header Sizing**: Velocity and Mach number calculations

In [ ]:
# ============================================================
# Example 9a: Fire Case Depressurization (API 521)
# ============================================================
# Compare adiabatic vs fire-engulfed vessel depressurization

# Create two identical vessels
gas = SystemSrkEos(320.0, 80.0)  # 47°C, 80 bar
gas.addComponent("methane", 0.85)
gas.addComponent("ethane", 0.10)
gas.addComponent("propane", 0.05)
gas.setMixingRule("classic")
ThermodynamicOperations(gas).TPflash()

# Vessel 1: Adiabatic (no fire)
feed1 = Stream("feed1", gas.clone())
feed1.run()
vessel_adiabatic = VesselDepressurization("Adiabatic Case", feed1)
vessel_adiabatic.setVesselGeometry(5.0, 2.0, VesselOrientation.HORIZONTAL)
vessel_adiabatic.setVesselProperties(0.025, 7850, 500, 45)  # 25mm carbon steel
vessel_adiabatic.setOrificeDiameter(0.035)  # 35mm orifice
vessel_adiabatic.setCalculationType(CalculationType.ENERGY_BALANCE)
vessel_adiabatic.setHeatTransferType(HeatTransferType.ADIABATIC)
vessel_adiabatic.run()

# Vessel 2: Fire case (pool fire per API 521)
feed2 = Stream("feed2", gas.clone())
feed2.run()
vessel_fire = VesselDepressurization("Fire Case (API 521)", feed2)
vessel_fire.setVesselGeometry(5.0, 2.0, VesselOrientation.HORIZONTAL)
vessel_fire.setVesselProperties(0.025, 7850, 500, 45)
vessel_fire.setOrificeDiameter(0.035)
vessel_fire.setCalculationType(CalculationType.ENERGY_BALANCE)
vessel_fire.setFireCase(True)
vessel_fire.setFireHeatFlux(25.0, "kW/m2")  # Pool fire with adequate drainage
vessel_fire.setWettedSurfaceFraction(0.5)  # 50% of vessel wetted
vessel_fire.run()

print(f"Fire heat input: {vessel_fire.getFireHeatInput('kW'):.1f} kW")
print(f"Fire heat input: {vessel_fire.getFireHeatInput('MW'):.3f} MW")

# Simulate both cases
dt = 0.5
end_time = 600.0

time_data = [0]
P_adiabatic = [vessel_adiabatic.getPressure("bar")]
T_adiabatic = [vessel_adiabatic.getTemperature("C")]
P_fire = [vessel_fire.getPressure("bar")]
T_fire = [vessel_fire.getTemperature("C")]

uuid1 = UUID.randomUUID()
uuid2 = UUID.randomUUID()

print("\nSimulating adiabatic and fire cases...")
for t in np.arange(dt, end_time + dt, dt):
    vessel_adiabatic.runTransient(dt, uuid1)
    vessel_fire.runTransient(dt, uuid2)
    
    if t % 30 == 0:
        time_data.append(t)
        P_adiabatic.append(vessel_adiabatic.getPressure("bar"))
        T_adiabatic.append(vessel_adiabatic.getTemperature("C"))
        P_fire.append(vessel_fire.getPressure("bar"))
        T_fire.append(vessel_fire.getTemperature("C"))

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(time_data, P_adiabatic, 'b-', linewidth=2, label='Adiabatic')
axes[0].plot(time_data, P_fire, 'r-', linewidth=2, label='Fire Case')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Pressure [bar]')
axes[0].set_title('Pressure Comparison: Adiabatic vs Fire')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_data, T_adiabatic, 'b-', linewidth=2, label='Adiabatic')
axes[1].plot(time_data, T_fire, 'r-', linewidth=2, label='Fire Case')
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='0°C')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Temperature [°C]')
axes[1].set_title('Temperature Comparison: Adiabatic vs Fire')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n=== Summary ===")
print(f"{'Case':<20} {'Final P':<12} {'Final T':<12} {'Min T':<12}")
print(f"{'Adiabatic':<20} {P_adiabatic[-1]:.1f} bar{'':<4} {T_adiabatic[-1]:.1f} °C{'':<4} {vessel_adiabatic.getMinimumTemperatureReached('C'):.1f} °C")
print(f"{'Fire (25 kW/m²)':<20} {P_fire[-1]:.1f} bar{'':<4} {T_fire[-1]:.1f} °C{'':<4} {vessel_fire.getMinimumTemperatureReached('C'):.1f} °C")

In [ ]:
# ============================================================
# Example 9b: Valve Opening Dynamics
# ============================================================
# Compare instant opening vs gradual valve opening

gas = SystemSrkEos(300.0, 60.0)
gas.addComponent("nitrogen", 1.0)
gas.setMixingRule("classic")
ThermodynamicOperations(gas).TPflash()

# Case 1: Instant valve opening
feed1 = Stream("instant", gas.clone())
feed1.run()
vessel_instant = VesselDepressurization("Instant Opening", feed1)
vessel_instant.setVolume(5.0)
vessel_instant.setOrificeDiameter(0.03)
vessel_instant.setCalculationType(CalculationType.ISOTHERMAL)
vessel_instant.run()

# Case 2: 5-second valve opening
feed2 = Stream("gradual", gas.clone())
feed2.run()
vessel_gradual = VesselDepressurization("5s Opening Time", feed2)
vessel_gradual.setVolume(5.0)
vessel_gradual.setOrificeDiameter(0.03)
vessel_gradual.setCalculationType(CalculationType.ISOTHERMAL)
vessel_gradual.setValveOpeningTime(5.0)  # 5 seconds to fully open
vessel_gradual.run()

# Case 3: 15-second valve opening (slow ESD valve)
feed3 = Stream("slow", gas.clone())
feed3.run()
vessel_slow = VesselDepressurization("15s Opening Time", feed3)
vessel_slow.setVolume(5.0)
vessel_slow.setOrificeDiameter(0.03)
vessel_slow.setCalculationType(CalculationType.ISOTHERMAL)
vessel_slow.setValveOpeningTime(15.0)  # 15 seconds
vessel_slow.run()

# Simulate
dt = 0.25
end_time = 60.0

time_data = [0]
P_instant = [vessel_instant.getPressure("bar")]
P_gradual = [vessel_gradual.getPressure("bar")]
P_slow = [vessel_slow.getPressure("bar")]
mdot_instant = [0]
mdot_gradual = [0]
mdot_slow = [0]

uuid1 = UUID.randomUUID()
uuid2 = UUID.randomUUID()
uuid3 = UUID.randomUUID()

for t in np.arange(dt, end_time + dt, dt):
    vessel_instant.runTransient(dt, uuid1)
    vessel_gradual.runTransient(dt, uuid2)
    vessel_slow.runTransient(dt, uuid3)
    
    if t % 2 == 0:
        time_data.append(t)
        P_instant.append(vessel_instant.getPressure("bar"))
        P_gradual.append(vessel_gradual.getPressure("bar"))
        P_slow.append(vessel_slow.getPressure("bar"))
        mdot_instant.append(vessel_instant.getDischargeRate("kg/s"))
        mdot_gradual.append(vessel_gradual.getDischargeRate("kg/s"))
        mdot_slow.append(vessel_slow.getDischargeRate("kg/s"))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(time_data, mdot_instant, 'b-', linewidth=2, label='Instant')
axes[0].plot(time_data, mdot_gradual, 'g-', linewidth=2, label='5s opening')
axes[0].plot(time_data, mdot_slow, 'r-', linewidth=2, label='15s opening')
axes[0].axvline(x=5, color='g', linestyle='--', alpha=0.5)
axes[0].axvline(x=15, color='r', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Mass Flow [kg/s]')
axes[0].set_title('Discharge Rate with Valve Opening Dynamics')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_data, P_instant, 'b-', linewidth=2, label='Instant')
axes[1].plot(time_data, P_gradual, 'g-', linewidth=2, label='5s opening')
axes[1].plot(time_data, P_slow, 'r-', linewidth=2, label='15s opening')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Pressure [bar]')
axes[1].set_title('Pressure Profile with Valve Opening Dynamics')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Peak discharge rates:")
print(f"  Instant opening: {vessel_instant.getPeakDischargeRate('kg/s'):.2f} kg/s")
print(f"  5s opening:      {vessel_gradual.getPeakDischargeRate('kg/s'):.2f} kg/s")
print(f"  15s opening:     {vessel_slow.getPeakDischargeRate('kg/s'):.2f} kg/s")

In [ ]:
# ============================================================
# Example 9c: Data Export (CSV and JSON)
# ============================================================

# Run a short simulation
gas = SystemSrkEos(300.0, 50.0)
gas.addComponent("methane", 0.95)
gas.addComponent("ethane", 0.05)
gas.setMixingRule("classic")
ThermodynamicOperations(gas).TPflash()

feed = Stream("export_demo", gas)
feed.run()

vessel = VesselDepressurization("Export Demo", feed)
vessel.setVolume(2.0)
vessel.setOrificeDiameter(0.025)
vessel.setCalculationType(CalculationType.ISOTHERMAL)
vessel.run()

# Run transient
uuid = UUID.randomUUID()
for t in range(60):
    vessel.runTransient(1.0, uuid)

# Export to CSV
csv_data = vessel.exportToCSV()
print("=== CSV Export (first 5 rows) ===")
for i, line in enumerate(csv_data.split('\n')[:6]):
    print(line)

print("\n")

# Export to JSON
json_data = vessel.exportToJSON()
print("=== JSON Export (first 15 lines) ===")
for i, line in enumerate(json_data.split('\n')[:15]):
    print(line)

print("\n... (data continues)")
print(f"\nTotal data points: {len(vessel.getTimeHistory())}")

# The CSV and JSON strings can be written to files:
# with open('blowdown_results.csv', 'w') as f:
#     f.write(csv_data)
# with open('blowdown_results.json', 'w') as f:
#     f.write(json_data)

In [ ]:
# ============================================================
# Example 9d: Flare Header Sizing
# ============================================================

# Calculate flare header velocity and Mach number for different header sizes

gas = SystemSrkEos(300.0, 70.0)
gas.addComponent("methane", 0.80)
gas.addComponent("ethane", 0.15)
gas.addComponent("propane", 0.05)
gas.setMixingRule("classic")
ThermodynamicOperations(gas).TPflash()

feed = Stream("header_sizing", gas)
feed.run()

vessel = VesselDepressurization("Header Sizing", feed)
vessel.setVolume(20.0)  # Large vessel
vessel.setOrificeDiameter(0.05)  # 50mm orifice
vessel.setCalculationType(CalculationType.ISOTHERMAL)
vessel.run()

# Run a few steps to get representative flow
uuid = UUID.randomUUID()
vessel.runTransient(1.0, uuid)

print("=== Flare Header Sizing at Peak Flow ===")
print(f"Peak discharge rate: {vessel.getPeakDischargeRate('kg/s'):.2f} kg/s")
print(f"                   : {vessel.getPeakDischargeRate('kg/hr'):.0f} kg/hr")
print()

# Check different header sizes
header_sizes = [
    (0.1, '4"'),
    (0.15, '6"'),
    (0.2, '8"'),
    (0.25, '10"'),
    (0.3, '12"'),
    (0.4, '16"'),
]

print(f"{'Header Size':<15} {'Velocity':<15} {'Mach':<10} {'Status':<20}")
print("-" * 60)

for diameter, name in header_sizes:
    velocity = vessel.getFlareHeaderVelocity(diameter, "m/s")
    mach = vessel.getFlareHeaderMach(diameter)
    
    if mach > 0.6:
        status = "⚠️  Too high (>0.6 Mach)"
    elif mach > 0.3:
        status = "⚡ High (noise/vibration)"
    else:
        status = "✓ OK"
    
    print(f"{name:<15} {velocity:>8.1f} m/s{'':<4} {mach:>6.3f}{'':<4} {status}")

print()
print("Design guidelines per API 521:")
print("  - Normal: Mach < 0.3 (low noise, minimal vibration)")
print("  - Maximum: Mach < 0.6-0.7 (emergency relief only)")
print("  - Velocity typically < 150 m/s for flare headers")

In [ ]:
# ============================================================
# Example 9e: Liquid Rainout Detection
# ============================================================
# Check for two-phase flow and liquid carryover in flare headers

# Create a rich gas that may condense during blowdown
gas = SystemSrkEos(290.0, 90.0)  # High pressure, moderate temp
gas.addComponent("methane", 0.60)
gas.addComponent("ethane", 0.15)
gas.addComponent("propane", 0.15)
gas.addComponent("n-butane", 0.07)
gas.addComponent("n-pentane", 0.03)
gas.setMixingRule("classic")
ThermodynamicOperations(gas).TPflash()

feed = Stream("rich_gas", gas)
feed.run()

vessel = VesselDepressurization("Rich Gas Blowdown", feed)
vessel.setVolume(10.0)
vessel.setOrificeDiameter(0.04)
vessel.setCalculationType(CalculationType.ENERGY_BALANCE)
vessel.setHeatTransferType(HeatTransferType.ADIABATIC)
vessel.run()

# Track liquid rainout during blowdown
dt = 1.0
end_time = 300.0

time_data = [0]
liquid_frac = [vessel.getOutletLiquidFraction()]
has_rainout = [vessel.hasLiquidRainout()]
pressure_data = [vessel.getPressure("bar")]
temp_data = [vessel.getTemperature("C")]

uuid = UUID.randomUUID()
print("Simulating rich gas blowdown (checking for liquid condensation)...")

for t in np.arange(dt, end_time + dt, dt):
    vessel.runTransient(dt, uuid)
    
    if t % 15 == 0:
        time_data.append(t)
        liquid_frac.append(vessel.getOutletLiquidFraction())
        has_rainout.append(vessel.hasLiquidRainout())
        pressure_data.append(vessel.getPressure("bar"))
        temp_data.append(vessel.getTemperature("C"))

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pressure
axes[0, 0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Pressure [bar]')
axes[0, 0].set_title('Vessel Pressure')
axes[0, 0].grid(True, alpha=0.3)

# Temperature
axes[0, 1].plot(time_data, temp_data, 'r-', linewidth=2)
axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Temperature [°C]')
axes[0, 1].set_title('Fluid Temperature')
axes[0, 1].grid(True, alpha=0.3)

# Liquid fraction in outlet
axes[1, 0].plot(time_data, [f*100 for f in liquid_frac], 'g-', linewidth=2)
axes[1, 0].axhline(y=10, color='orange', linestyle='--', label='10% threshold')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Liquid Mass Fraction [%]')
axes[1, 0].set_title('Liquid in Outlet Stream (Rainout Risk)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Phase status
axes[1, 1].plot(time_data, [1 if r else 0 for r in has_rainout], 'purple', linewidth=2, drawstyle='steps-post')
axes[1, 1].set_xlabel('Time [s]')
axes[1, 1].set_ylabel('Has Liquid')
axes[1, 1].set_title('Two-Phase Flow Detection')
axes[1, 1].set_ylim(-0.1, 1.1)
axes[1, 1].set_yticks([0, 1])
axes[1, 1].set_yticklabels(['Gas Only', 'Two-Phase'])
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
peak_liquid = max(liquid_frac)
print(f"\n=== Liquid Rainout Assessment ===")
print(f"Peak liquid fraction in outlet: {peak_liquid*100:.1f}%")
print(f"Minimum temperature reached: {vessel.getMinimumTemperatureReached('C'):.1f}°C")

if peak_liquid > 0.10:
    print("\n⚠️  WARNING: >10% liquid detected in outlet")
    print("   Consider adding knockout drum before flare header")
elif peak_liquid > 0.01:
    print("\n⚡ CAUTION: Some liquid detected (<10%)")
    print("   Monitor for liquid accumulation in flare header")
else:
    print("\n✓ No significant liquid carryover detected")

## Example 10: Flow Assurance Risk Assessment

This example demonstrates hydrate formation and CO2 freezing (dry ice) risk evaluation during blowdown.

### Flow Assurance Risks During Blowdown

**Hydrate Formation:**
- When gas contains water and hydrocarbons, hydrate crystals can form at low temperatures
- Hydrates can plug relief lines and valves
- Risk increases as temperature drops during depressurization

**CO2 Freezing (Dry Ice):**
- CO2 can form solid dry ice below its sublimation/freezing curve
- At 1 bar: -78.5°C, at 5.18 bar (triple point): -56.6°C
- Can cause equipment damage and flow blockages

In [ ]:
# Example 10a: Hydrate Risk During Natural Gas Blowdown
# ============================================================

# Create a wet natural gas system (with water)
gas = SystemSrkCPAstatoil(288.0, 100.0)  # 15°C, 100 bar
gas.addComponent("methane", 0.80)
gas.addComponent("ethane", 0.08)
gas.addComponent("propane", 0.05)
gas.addComponent("n-butane", 0.02)
gas.addComponent("CO2", 0.03)
gas.addComponent("water", 0.02)  # Wet gas!
gas.setMixingRule(10)  # CPA mixing rule
gas.createDatabase(True)

feed = Stream("feed", gas)
feed.setFlowRate(0.0, "kg/hr")
feed.run()

# Create vessel - offshore separator scenario
vessel = VesselDepressurization("Wet Gas Separator", feed)
vessel.setVolume(15.0)  # 15 m³ separator
vessel.setOrificeDiameter(0.04)  # 40mm relief valve
vessel.setBackPressure(5.0)  # 5 bar flare header
vessel.setCalculationType(VesselDepressurization.CalculationType.ADIABATIC)
vessel.run()

# Check initial hydrate conditions
print("=== Initial Hydrate Risk Assessment ===")
print(f"Initial pressure: {vessel.getPressure('bar'):.1f} bar")
print(f"Initial temperature: {vessel.getTemperature('C'):.1f}°C")

try:
    hydrate_temp = vessel.getHydrateFormationTemperature("C")
    if hydrate_temp > -999:
        print(f"Hydrate formation temperature: {hydrate_temp:.1f}°C")
        subcooling = vessel.getHydrateSubcooling("C")
        if subcooling > 0:
            print(f"⚠️  Hydrate risk! Subcooling: {subcooling:.1f}°C")
        else:
            print(f"✓ No hydrate risk. Margin: {-subcooling:.1f}°C")
except Exception as e:
    print(f"Hydrate calculation not available: {e}")

# Run blowdown simulation tracking hydrate risk
print("\n=== Simulating Blowdown with Hydrate Monitoring ===")
dt = 0.5
end_time = 300.0
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel.getPressure("bar")]
temp_data = [vessel.getTemperature("C")]
has_hydrate_risk = [vessel.hasHydrateRisk()]

for t in np.arange(dt, end_time + dt, dt):
    vessel.runTransient(dt, uuid)
    
    if t % 10 == 0:
        time_data.append(t)
        pressure_data.append(vessel.getPressure("bar"))
        temp_data.append(vessel.getTemperature("C"))
        has_hydrate_risk.append(vessel.hasHydrateRisk())

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pressure profile
axes[0, 0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Pressure [bar]')
axes[0, 0].set_title('Pressure During Blowdown')
axes[0, 0].grid(True, alpha=0.3)

# Temperature vs hydrate curve
axes[0, 1].plot(time_data, temp_data, 'r-', linewidth=2, label='Fluid Temperature')
axes[0, 1].axhline(y=0, color='blue', linestyle='--', alpha=0.5, label='0°C reference')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Temperature [°C]')
axes[0, 1].set_title('Temperature Profile')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Hydrate risk indicator
risk_numeric = [1 if r else 0 for r in has_hydrate_risk]
axes[1, 0].fill_between(time_data, risk_numeric, alpha=0.3, color='red', step='post')
axes[1, 0].plot(time_data, risk_numeric, 'r-', linewidth=2, drawstyle='steps-post')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Risk Present')
axes[1, 0].set_title('Hydrate Formation Risk')
axes[1, 0].set_ylim(-0.1, 1.1)
axes[1, 0].set_yticks([0, 1])
axes[1, 0].set_yticklabels(['No Risk', 'RISK'])
axes[1, 0].grid(True, alpha=0.3)

# P-T trajectory
axes[1, 1].scatter(temp_data, pressure_data, c=time_data, cmap='viridis', s=30)
axes[1, 1].plot(temp_data, pressure_data, 'k--', alpha=0.3)
axes[1, 1].set_xlabel('Temperature [°C]')
axes[1, 1].set_ylabel('Pressure [bar]')
axes[1, 1].set_title('P-T Path During Blowdown')
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Time [s]')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print(f"\n=== Hydrate Risk Summary ===")
print(f"Final pressure: {pressure_data[-1]:.1f} bar")
print(f"Minimum temperature: {vessel.getMinimumTemperatureReached('C'):.1f}°C")

# Check max subcooling during blowdown
max_subcooling = vessel.getMaxHydrateSubcoolingDuringBlowdown()
if max_subcooling > 0:
    print(f"\n⚠️  Maximum hydrate subcooling: {max_subcooling:.1f}°C")
    print("   Consider:")
    print("   - Injecting methanol/MEG")
    print("   - Using heating during depressurization")
    print("   - Installing hydrate inhibitor injection point")
else:
    print(f"\n✓ No hydrate risk during blowdown")

In [ ]:
# Example 10b: CO2 Freezing Risk Assessment
# ============================================================
# High-CO2 gas (e.g., carbon capture application)

gas_co2 = SystemSrkEos(293.0, 80.0)  # 20°C, 80 bar
gas_co2.addComponent("CO2", 0.90)
gas_co2.addComponent("methane", 0.08)
gas_co2.addComponent("nitrogen", 0.02)
gas_co2.setMixingRule("classic")
gas_co2.createDatabase(True)

feed_co2 = Stream("CO2 rich feed", gas_co2)
feed_co2.setFlowRate(0.0, "kg/hr")
feed_co2.run()

# Create vessel - CO2 storage/transport scenario
vessel_co2 = VesselDepressurization("CO2 Vessel", feed_co2)
vessel_co2.setVolume(10.0)  # 10 m³
vessel_co2.setOrificeDiameter(0.03)  # 30mm orifice
vessel_co2.setBackPressure(1.0)  # Atmospheric
vessel_co2.setCalculationType(VesselDepressurization.CalculationType.ADIABATIC)
vessel_co2.run()

# Check initial CO2 freezing conditions
print("=== Initial CO2 Freezing Risk Assessment ===")
print(f"Initial pressure: {vessel_co2.getPressure('bar'):.1f} bar")
print(f"Initial temperature: {vessel_co2.getTemperature('C'):.1f}°C")

co2_freeze_temp = vessel_co2.getCO2FreezingTemperature("C")
if co2_freeze_temp > -999:
    print(f"CO2 freezing temperature at this pressure: {co2_freeze_temp:.1f}°C")
    if vessel_co2.hasCO2FreezingRisk():
        print("⚠️  CO2 freezing risk exists!")
    else:
        margin = vessel_co2.getCO2FreezingSubcooling("C")
        print(f"✓ No CO2 freezing risk. Margin: {-margin:.1f}°C")

# Run blowdown simulation
print("\n=== Simulating CO2 Blowdown ===")
dt = 0.2
end_time = 180.0
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_co2.getPressure("bar")]
temp_data = [vessel_co2.getTemperature("C")]
co2_freeze_line = [vessel_co2.getCO2FreezingTemperature("C")]
has_co2_freeze = [vessel_co2.hasCO2FreezingRisk()]

for t in np.arange(dt, end_time + dt, dt):
    vessel_co2.runTransient(dt, uuid)
    
    if t % 5 == 0:
        time_data.append(t)
        pressure_data.append(vessel_co2.getPressure("bar"))
        temp_data.append(vessel_co2.getTemperature("C"))
        co2_freeze_line.append(vessel_co2.getCO2FreezingTemperature("C"))
        has_co2_freeze.append(vessel_co2.hasCO2FreezingRisk())

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pressure profile
axes[0, 0].plot(time_data, pressure_data, 'b-', linewidth=2)
axes[0, 0].axhline(y=5.18, color='purple', linestyle='--', alpha=0.7, label='CO2 Triple Point (5.18 bar)')
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Pressure [bar]')
axes[0, 0].set_title('Pressure During Blowdown')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Temperature vs CO2 freeze curve
axes[0, 1].plot(time_data, temp_data, 'r-', linewidth=2, label='Fluid Temperature')
axes[0, 1].plot(time_data, co2_freeze_line, 'b--', linewidth=2, label='CO2 Freeze Line')
axes[0, 1].fill_between(time_data, co2_freeze_line, [-100]*len(time_data), 
                         alpha=0.2, color='blue', label='Solid CO2 Region')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Temperature [°C]')
axes[0, 1].set_title('Temperature vs CO2 Freezing Curve')
axes[0, 1].legend()
axes[0, 1].set_ylim([-90, 30])
axes[0, 1].grid(True, alpha=0.3)

# CO2 freezing risk indicator
risk_numeric = [1 if r else 0 for r in has_co2_freeze]
axes[1, 0].fill_between(time_data, risk_numeric, alpha=0.3, color='blue', step='post')
axes[1, 0].plot(time_data, risk_numeric, 'b-', linewidth=2, drawstyle='steps-post')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Risk Present')
axes[1, 0].set_title('CO2 Freezing (Dry Ice) Risk')
axes[1, 0].set_ylim(-0.1, 1.1)
axes[1, 0].set_yticks([0, 1])
axes[1, 0].set_yticklabels(['No Risk', 'DRY ICE RISK'])
axes[1, 0].grid(True, alpha=0.3)

# P-T phase diagram with trajectory
# Create CO2 phase boundaries
p_sublim = np.linspace(0.1, 5.18, 50)
t_sublim = [1512.0 / (15.96 - np.log(max(p, 0.001))) - 273.15 for p in p_sublim]
p_melt = np.linspace(5.18, 100, 50)
t_melt = [216.55 + 0.02 * (p - 5.18) - 273.15 for p in p_melt]

axes[1, 1].plot(t_sublim, p_sublim, 'b-', linewidth=2, label='Sublimation Line')
axes[1, 1].plot(t_melt, p_melt, 'b-', linewidth=2, label='Melting Line')
axes[1, 1].plot(-56.6, 5.18, 'ko', markersize=10, label='Triple Point', zorder=5)
axes[1, 1].scatter(temp_data, pressure_data, c=time_data, cmap='Reds', s=50, zorder=4)
axes[1, 1].plot(temp_data, pressure_data, 'r--', alpha=0.5, linewidth=1)
axes[1, 1].fill_betweenx(p_sublim, t_sublim, -100, alpha=0.15, color='blue')
axes[1, 1].fill_betweenx(p_melt, t_melt, -100, alpha=0.15, color='blue')
axes[1, 1].set_xlabel('Temperature [°C]')
axes[1, 1].set_ylabel('Pressure [bar]')
axes[1, 1].set_title('CO2 Phase Diagram with Blowdown Path')
axes[1, 1].set_xlim([-90, 30])
axes[1, 1].set_ylim([0, 100])
axes[1, 1].legend(loc='upper left')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].text(-75, 2, 'SOLID', fontsize=10, color='blue')
axes[1, 1].text(-40, 50, 'LIQUID', fontsize=10, color='green')
axes[1, 1].text(0, 30, 'GAS', fontsize=10, color='red')

plt.tight_layout()
plt.show()

# Summary
print(f"\n=== CO2 Freezing Risk Summary ===")
print(f"Final pressure: {pressure_data[-1]:.1f} bar")
print(f"Minimum temperature: {vessel_co2.getMinimumTemperatureReached('C'):.1f}°C")
print(f"CO2 freezing point at final pressure: {co2_freeze_line[-1]:.1f}°C")

if any(has_co2_freeze):
    print("\n⚠️  CO2 FREEZING RISK DETECTED!")
    print("   Dry ice may form and cause:")
    print("   - Valve blockage")
    print("   - Pipe damage")
    print("   - Pressure build-up")
    print("   Consider slower depressurization rate")
else:
    min_margin = min([t - f for t, f in zip(temp_data, co2_freeze_line)])
    print(f"\n✓ No CO2 freezing risk. Minimum margin: {min_margin:.1f}°C")

In [ ]:
# Example 10c: Comprehensive Flow Assurance Risk Report
# ============================================================

# Create a challenging scenario with multiple risks
gas_complex = SystemSrkCPAstatoil(280.0, 120.0)  # 7°C, 120 bar
gas_complex.addComponent("methane", 0.70)
gas_complex.addComponent("ethane", 0.10)
gas_complex.addComponent("propane", 0.03)
gas_complex.addComponent("CO2", 0.10)
gas_complex.addComponent("H2S", 0.02)
gas_complex.addComponent("water", 0.05)  # Saturated with water
gas_complex.setMixingRule(10)
gas_complex.createDatabase(True)

feed_complex = Stream("complex feed", gas_complex)
feed_complex.setFlowRate(0.0, "kg/hr")
feed_complex.run()

# Create vessel
vessel_complex = VesselDepressurization("Sour Gas Separator", feed_complex)
vessel_complex.setVolume(25.0)  # Large separator
vessel_complex.setOrificeDiameter(0.05)  # 50mm relief
vessel_complex.setBackPressure(3.0)  # Flare header
vessel_complex.setCalculationType(VesselDepressurization.CalculationType.ADIABATIC)

# Set MDMT (Minimum Design Metal Temperature)
vessel_complex.setMinimumDesignMetalTemperature(-46.0, "C")  # Carbon steel limit
vessel_complex.run()

print("=== Comprehensive Flow Assurance Risk Assessment ===\n")
print(f"Vessel: {vessel_complex.getName()}")
print(f"Volume: 25 m³")
print(f"Initial conditions: {vessel_complex.getPressure('bar'):.1f} bar, {vessel_complex.getTemperature('C'):.1f}°C")
print(f"MDMT limit: -46°C")

# Run simulation and collect comprehensive risk data
dt = 0.5
end_time = 600.0  # 10 minutes
uuid = UUID.randomUUID()

time_data = [0]
pressure_data = [vessel_complex.getPressure("bar")]
temp_data = [vessel_complex.getTemperature("C")]
liquid_frac = [vessel_complex.getOutletLiquidFraction()]

for t in np.arange(dt, end_time + dt, dt):
    vessel_complex.runTransient(dt, uuid)
    
    if t % 15 == 0:
        time_data.append(t)
        pressure_data.append(vessel_complex.getPressure("bar"))
        temp_data.append(vessel_complex.getTemperature("C"))
        liquid_frac.append(vessel_complex.getOutletLiquidFraction())

# Generate comprehensive risk report
print("\n" + "="*60)
print("FLOW ASSURANCE RISK REPORT")
print("="*60)

risks = vessel_complex.assessFlowAssuranceRisks()

print("\n📋 RISK CATEGORIES:")
print("-"*40)

for risk_type in ["HYDRATE", "CO2_FREEZING", "MDMT", "LIQUID_RAINOUT"]:
    assessment = str(risks.get(risk_type))
    
    # Determine risk icon
    if "CRITICAL" in assessment.upper() or "HIGH" in assessment.upper():
        icon = "🔴"
    elif "WARNING" in assessment.upper() or "MEDIUM" in assessment.upper():
        icon = "🟡"
    elif "OK" in assessment.upper() or "LOW" in assessment.upper():
        icon = "🟢"
    else:
        icon = "⚪"
    
    print(f"\n{icon} {risk_type}:")
    print(f"   {assessment}")

print("\n" + "="*60)
print("SIMULATION SUMMARY")
print("="*60)

print(f"\n📊 Pressure Range: {max(pressure_data):.1f} → {min(pressure_data):.1f} bar")
print(f"🌡️  Temperature Range: {max(temp_data):.1f} → {min(temp_data):.1f}°C")
print(f"💧 Max Liquid Fraction: {max(liquid_frac)*100:.2f}%")
print(f"⏱️  Blowdown Time: {end_time/60:.1f} minutes")

# Plot comprehensive overview
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# P-T trajectory with risk zones
axes[0, 0].plot(temp_data, pressure_data, 'ko-', markersize=3, linewidth=1)
axes[0, 0].plot(temp_data[0], pressure_data[0], 'go', markersize=12, label='Start', zorder=5)
axes[0, 0].plot(temp_data[-1], pressure_data[-1], 'ro', markersize=12, label='End', zorder=5)
axes[0, 0].axvline(x=-46, color='blue', linestyle='--', alpha=0.7, label='MDMT (-46°C)')
axes[0, 0].axvline(x=0, color='cyan', linestyle=':', alpha=0.5, label='0°C')
axes[0, 0].fill_betweenx([0, 150], [-100, -100], [-46, -46], alpha=0.15, color='blue')
axes[0, 0].set_xlabel('Temperature [°C]')
axes[0, 0].set_ylabel('Pressure [bar]')
axes[0, 0].set_title('P-T Path with Risk Zones')
axes[0, 0].legend(loc='upper right')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xlim([min(temp_data)-10, max(temp_data)+10])

# Time profiles
axes[0, 1].plot(time_data, temp_data, 'r-', linewidth=2, label='Temperature')
ax2 = axes[0, 1].twinx()
ax2.plot(time_data, pressure_data, 'b-', linewidth=2, label='Pressure')
axes[0, 1].axhline(y=-46, color='purple', linestyle='--', alpha=0.7)
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Temperature [°C]', color='red')
ax2.set_ylabel('Pressure [bar]', color='blue')
axes[0, 1].set_title('Temperature and Pressure vs Time')
axes[0, 1].grid(True, alpha=0.3)

# Liquid carryover
axes[1, 0].fill_between(time_data, [f*100 for f in liquid_frac], alpha=0.3, color='green')
axes[1, 0].plot(time_data, [f*100 for f in liquid_frac], 'g-', linewidth=2)
axes[1, 0].axhline(y=10, color='orange', linestyle='--', label='10% Warning')
axes[1, 0].axhline(y=20, color='red', linestyle='--', label='20% Critical')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('Liquid Mass Fraction [%]')
axes[1, 0].set_title('Liquid Carryover to Flare')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Risk summary gauge chart (simplified)
risk_categories = ['Hydrate', 'CO2 Freeze', 'MDMT', 'Rainout']
# Estimate risk levels (0-3 scale)
risk_levels = []
for r in ["HYDRATE", "CO2_FREEZING", "MDMT", "LIQUID_RAINOUT"]:
    assessment = str(risks.get(r)).upper()
    if "CRITICAL" in assessment or "HIGH" in assessment:
        risk_levels.append(3)
    elif "WARNING" in assessment or "MEDIUM" in assessment:
        risk_levels.append(2)
    elif "LOW" in assessment or "RISK" in assessment:
        risk_levels.append(1)
    else:
        risk_levels.append(0)

colors = ['green' if r==0 else 'yellow' if r<=1 else 'orange' if r<=2 else 'red' for r in risk_levels]
bars = axes[1, 1].bar(risk_categories, risk_levels, color=colors, edgecolor='black', linewidth=2)
axes[1, 1].set_ylim([0, 4])
axes[1, 1].set_ylabel('Risk Level')
axes[1, 1].set_title('Risk Summary')
axes[1, 1].set_yticks([0, 1, 2, 3])
axes[1, 1].set_yticklabels(['OK', 'Low', 'Medium', 'High'])
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Flow Assurance Assessment Complete")